# Logical Elements

In [50]:
class Literal(object):
    """Represents a literal in a clause.
    
    Parameters
    ----------
    symbol : Symbol object
        A link to the corresponding symbol/variable of the literal.
    value : True, False or "free"
        The initial value of the literal. "free" means it's unassigned.
    sign : bool
        Indicates whether the literal is a variable or the negation
        of a variable.
        
    """
    def __init__(self, symbol, value, sign):
        self.symbol = symbol
        self.value = value
        self.sign = sign
    
    def __nonzero__(self):
        return self.value is True
        
class Symbol(object):
    """Represents a symbol.
    
    Parameters
    ----------
    name : str
        The name of the symbol.
        
    Attributes
    ----------
    pos : Literal object
        A link to the positive literal representation.
    neg : Literal object
        A link to the negative literal representation.
    value : True, False or "free"
        The value that's currently assigned to this symbol.
        
    """
    def __init__(self, name):
        self.name = name
        self.pos = Literal(self, "free", True)
        self.neg = Literal(self, "free", False)
        self.value = "free"
        
    def __repr__(self):
        return self.name
    
    def __neg__(self):
        return self.neg
    
    def set(self, value):
        # When assigning value to the symbol, automatically assign
        # values to linked literals.
        self.value = value
        self.pos.value = value
        self.neg.value = not value
        
    def unset(self):
        # When setting the symbol free, automatically set both linked
        # literals free.
        self.value = "free"
        self.pos.value = "free"
        self.neg.value = "free"
    
class Clause(object):
    """Represents a clause in CNF.
    
    Parameters
    ----------
    literals : list of Literal objects
        Contains the literals in this clause.

    Methods
    -------
    
    __nonzero__:
        Returns True if at least one literal is True. Otherwise returns 
        False. Returning False does not mean it's unsatisfied, only not
        yet satisfied.
    all_false :
        Returns True if this clause is evaluated to False (when all its
        literals have been assigned False). Otherwise returns False.
    get_free_lits:
        Returns a list of literals which are not yet assigned.
        
    """
    def __init__(self, literals):
        self.literals = literals
        
    def __nonzero__(self):
        return any(self.literals)
    
    def all_false(self):
        return all(x.value is False for x in self.literals)
    
    def get_free_lits(self):
        return [x for x in self.literals if x.value == "free"]
               

# DPLL Main Body

In [48]:
import operator

def clean_up(KB):
    """ Change KB's symbols to positive literals.
    Remove redundant literals(e.g. A or A) or clauses (e.g. A or not A).
    """
    for i in range(len(KB)):
        for j in range(len(KB[i])):
            if isinstance(KB[i][j], Symbol):
                KB[i][j] = KB[i][j].pos
           
    new_KB = []
    for clause in KB:
        redundant = False
        appeared = {}
        for literal in clause:
            if literal.symbol in appeared:
                if appeared[literal.symbol] is not literal:
                    redundant = True
                    break
            else:
                appeared[literal.symbol] = literal
        if not redundant:
            new_KB.append(appeared.values())
                      
    return new_KB

def preprocess(KB):
    """Clean up KB, instantiate Clause objects, and generate
    a unique set of symbols.
    
    Returns a list of Clause objects and a set of Symbol objects.
    """
    KB = clean_up(KB)
    clauses = []
    all_symbols = set()

    for each in KB:
        literals = []
        for lit in each:
            all_symbols.add(lit.symbol)
            literals.append(lit)
        clauses.append(Clause(literals))
    return clauses, all_symbols

def DPLL_Satisfiable(KB):
    def count_literals(unassigned, unsat_clauses):
        """
        This is a counting function that's helpful to both degree
        heuristics and pure symbol elimination.
        
        degree_count tracks the number of times each unassigned 
        symbol appears in a remaining unsatisfied clauses.
        
        lit_count tracks the number of times each unassigned symbol
        appears in a remaining unsatisfied clauses in positive and
        negative forms, respectively.
        """
        degree_count = {x:0 for x in unassigned}
        lit_count = {x:[0,0] for x in unassigned}#[pos_count, neg_count]
        for clause in unsat_clauses:
            # Each symbol can appear at most once in a clause
            # due to preprocessing
            for lit in clause.literals:
                x = lit.symbol
                if x in degree_count:# if symbol is in unassigned
                    degree_count[x] += 1
                    if lit.sign:#if literal is positive
                        lit_count[x][0] += 1
                    else:#else literal is negative
                        lit_count[x][1] += 1
        return degree_count, lit_count
        
    def pop_by_degree(unassigned, degree_count):
        """Pop an unassigned symbol using degree heuristic.
        Works by choosing an unassigned variable with the most edges
        on ther
        """
        # Get symbol with max degree count
        new_symbol = max(degree_count.iteritems(), 
                         key=operator.itemgetter(1))[0] 
        return new_symbol  
    
    def get_pure_symbol(lit_count):
        pure_symbols = []
        for x, count in lit_count.items():
            # If the symbol only appears positive
            if (count[0] > 0) and (count[1] == 0):
                pure_symbols.append((count[0], (x, True)))
                return x, True
            # If the symbol only appears negative
            if (count[0] == 0) and (count[1] > 0):
                pure_symbols.append((count[1], (x, False)))
                
        length = len(pure_symbols)
        if length == 0:
            return None, None
        if length == 1:
            return pure_symbols[0][1]
        
        # Return the symbol with maximum count of appearance
        return max(pure_symbols, key=operator.itemgetter(0))[1]
    
    def get_unit_clause(unsat_clauses):
        for clause in unsat_clauses:
            free_lits = clause.get_free_lits()
            # If there is only one free literal in clauses which are 
            # not yet satisfied, it's a unit clause
            if len(free_lits) == 1:
                symbol = free_lits[0].symbol
                sign = free_lits[0].sign
                return symbol, sign
        return None, None
    
    def recursive_DPLL(model, unassigned):
        """Recursively searches for a model (a possible set of 
        assignments of symbols) which satisfies all clauses.
        
        'unassigned' keeps track of symbols which have not been
        assigned a value.
        
        """
        def try_assign(symbol, value, model, unassigned):
            """
            Function that takes care of setting and unsetting relevant
            variables for next recursion, call the next recursion and 
            check if solution has been found.
            """
            assignment = (symbol, value)
            model.add(assignment) # add to model
            symbol.set(value) # update symbol value
            unassigned.remove(symbol) # pop from unassigned
            
            sat, sol = recursive_DPLL(model,unassigned)
            if sat:
                return sat, sol
            
            # Undo all changes
            model.remove(assignment)
            symbol.unset()
            unassigned.add(symbol)
            
            return False, None
            
        # If all clauses evaluate to True, solutions has been found    
        if all(clauses):
            return True, model
        # If at least one clause evaluates to False, stop the search
        if any(x.all_false() for x in clauses):
            return False, None
        
        # Keep tracks of unsat clauses. "unsat" here doesn't mean 
        # it evaluates to False, just that it's not satisfied yet.
        unsat_clauses = [clause for clause in clauses if not clause]
        # Get degree and literal counts for future actions.
        degree_count, lit_count = count_literals(unassigned, unsat_clauses)
        
        # Pure symbol elimination
        pure, sign = get_pure_symbol(lit_count)
        if pure is not None:
            return try_assign(pure, sign, model, unassigned)
        
        # Unit clause propagation
        unit, sign = get_unit_clause(unsat_clauses)
        if unit is not None:
            return try_assign(unit, sign, model, unassigned)
        
        # Chosse next symbol to assign using degree heuristics
        new_symbol = pop_by_degree(unassigned, degree_count)
        
        # Assign to True
        sat, sol = try_assign(new_symbol, True, model, unassigned)
        if sat:
            return sat, sol
        
        # Assign to False
        sat, sol = try_assign(new_symbol, False, model, unassigned)
        if sat:
            return sat, sol
        
        return False, None
                
    
    clauses, all_symbols = preprocess(KB)
    sat, sol = recursive_DPLL(set(), all_symbols.copy())
    if sat:
        # Printing requirement
        return True, {symbol:str(symbol.value).lower() 
                      for symbol in all_symbols}
    return sat, sol


# Testing Correctness against PycoSAT

In [52]:
from random import randint
import pycosat

def my_solver(cnf):
    """Format the CNF to the correct format for DPLL and run it.
    """
    vset = set().union(*cnf)
    nvar = max(max(vset), abs(min(vset)))
    symbols = {}
    for var in range(1, nvar+1):
        symbols[var] = Symbol(str(var))
    KB = []
    for clause in cnf:
        KB.append([symbols[i] if i>0 else -symbols[-i] for i in clause])
    satisfiable, model = DPLL_Satisfiable(KB)
    return satisfiable, model

def random_3sat(nvar, ratio=4.2):
    """A function that generate a random 3 sat instance with given
    clause-to-variable ratio. A ratio of 4.2 is empirically shown to
    give the hardest problems (Kirkpatrick & Selman, 1994).
    """
    sign = [1, -1]
    ncls = int(nvar * ratio)
    cnf = [] 
    for each_c in range(ncls):
        cnf.append([randint(1, nvar)*sign[randint(0,1)] for _ in range(3)])
    return cnf
    
def check(cnf):
    """Check whether my DPLL is correct:
    
    1. Checks whether my DPLL and pycosat both give "UNSAT"
    2. If both gives satisfying solutions, check whether solution
       from my DPLL is correct by plugging it back in to the CNF.
    """
    sat, sol = my_solver(cnf)
    ref_sol = pycosat.solve(cnf)
    if (ref_sol == "UNSAT") and (sat is False):
        return True
    elif (not isinstance(ref_sol, basestring)) and (sat is True):
        assignment = {int(x.name):(True if val=="true" else False) for x, val in sol.items()}
        neg_assign = {-x:(True if val is False else False) for x, val in assignment.items()}
        assignment.update(neg_assign)
        for clause in cnf:
            if not any(assignment[x] for x in clause):
                return False
        return True
    else:
        return False
        
def test(ntrials=100, nvar=25):
    """Test function that runs multiple trials with a designated number
    of variables in CNF.
    """
    for i in range(ntrials):
        cnf = random_3sat(nvar)
        if not check(cnf):
            print "Algorithm failed the test!"
            return
    print "Algorithm passed the test!"
        
test()

Algorithm passed the test!


-------------------
References

S. Kirkpatrick, B. Selman et al., “Critical behavior in the satisfiability
of random boolean expressions,” Science-AAAS-Weekly Paper
Edition-including Guide to Scientific Information, vol. 264, no. 5163,
pp. 1297–1300, 1994.

# Exercise 7.20 of Russell and Norvig

## Problem Description:

S1: A ⇔ (B∨E). 

S2: E ⇒ D. 

S3: C∧F ⇒ ¬B.

S4: E ⇒ B.

S5: B ⇒ F.

S6: B ⇒ C

## Converting to CNF:

S1: 

(A⇒(B∨E))∧((B∨E)⇒ A)

(¬A ∨ B∨E)∧(¬(B∨E) ∨ A)

(¬A ∨ B∨E)∧((¬B∧¬E) ∨ A)

(¬A ∨ B∨E)∧((¬B∨ A)∧(¬E∨ A))

Clauses: 

(¬A ∨ B∨E)

(¬B∨ A)

(¬E∨ A)
        
S2: E ⇒ D

(¬ E ∨ D)

S3: C∧F ⇒ ¬B

¬(C∧F) ∨ ¬B

(¬C∨¬F∨ ¬B)

S4: E ⇒ B

(¬ E ∨ B)

S5: B ⇒ F 

(¬ B ∨ F)

S6: B ⇒ C

(¬ B ∨ C)
    

## Converting to Correct Input Format

KB = [[-A, B, E], [-B, A], [-E, A], [-E, D], [-C, -F, -B], [-E, B], [-B, F], [-B, C]]

## Running the code

In [54]:
A = Symbol("A")
B = Symbol("B")
C = Symbol("C")
D = Symbol("D")
E = Symbol("E")
F = Symbol("F")
KB = [[-A, B, E], [-B, A], [-E, A], [-E, D], [-C, -F, -B], [-E, B], [-B, F], [-B, C]]
satisfiable, model = DPLL_Satisfiable(KB)
print satisfiable, model

True {A: 'false', C: 'free', F: 'free', D: 'true', B: 'false', E: 'false'}
